In [1]:
import pandas as pd
from scipy.stats import entropy

data = pd.read_csv("buy_computer.csv")
data.head()

,id,age,income,student,credit_rating,Buy_Computer
0,1,youth,high,no,fair,no
1,2,youth,high,no,excellent,no
2,3,middle_age,high,no,fair,yes
3,4,senior,medium,no,fair,yes
4,5,senior,low,yes,fair,yes


In [2]:
train_data = data.iloc[:14]
test_data = data.iloc[14:]

train_data

,id,age,income,student,credit_rating,Buy_Computer
0,1,youth,high,no,fair,no
1,2,youth,high,no,excellent,no
2,3,middle_age,high,no,fair,yes
3,4,senior,medium,no,fair,yes
4,5,senior,low,yes,fair,yes
5,6,senior,low,yes,excellent,no
6,7,middle_age,low,yes,excellent,yes
7,8,youth,medium,no,fair,no
8,9,youth,low,yes,fair,yes
9,10,senior,medium,yes,fair,yes


In [3]:
train_data["Buy_Computer"].value_counts()

yes    9
no     5
Name: Buy_Computer, dtype: int64

In [4]:
entropy_buy_computer = entropy([5/14, 9/14], base=2)
entropy_buy_computer

0.940285958670631

In [5]:
train_data.pivot_table(index='age',columns='Buy_Computer',aggfunc=lambda x: len(x))

credit_rating        id      income      student     
Buy_Computer            no  yes   no  yes     no  yes      no  yes
age                                                               
middle_age             NaN  4.0  NaN  4.0    NaN  4.0     NaN  4.0
senior                 2.0  3.0  2.0  3.0    2.0  3.0     2.0  3.0
youth                  3.0  2.0  3.0  2.0    3.0  2.0     3.0  2.0

In [6]:
entropy_buy_computer_age = (5/14)*entropy([3/5, 2/5], base=2)+(5/14)*entropy([2/5, 3/5], base=2)+(4/14)*entropy([0/4, 4/4], base=2)
entropy_buy_computer_age

0.693536138896192

In [7]:
train_data.pivot_table(index='income',columns='Buy_Computer',aggfunc=lambda x: len(x))

age     credit_rating     id     student    
Buy_Computer  no yes            no yes no yes      no yes
income                                                   
high           2   2             2   2  2   2       2   2
low            1   3             1   3  1   3       1   3
medium         2   4             2   4  2   4       2   4

In [8]:
entropy_buy_computer_income = (6/14)*entropy([2/6, 4/6], base=2)+(4/14)*entropy([1/4, 3/4], base=2)+(4/14)*entropy([2/4, 2/4], base=2)
entropy_buy_computer_income

0.9110633930116763

In [9]:
train_data.pivot_table(index='student',columns='Buy_Computer',aggfunc=lambda x: len(x))

age     credit_rating     id     income    
Buy_Computer  no yes            no yes no yes     no yes
student                                                 
no             4   3             4   3  4   3      4   3
yes            1   6             1   6  1   6      1   6

In [10]:
entropy_buy_computer_student = (7/14)*entropy([1/7, 6/7], base=2)+(7/14)*entropy([4/7, 3/7], base=2)
entropy_buy_computer_student

0.7884504573082894

In [11]:
train_data.pivot_table(index='credit_rating',columns='Buy_Computer',aggfunc=lambda x: len(x))

age     id     income     student    
Buy_Computer   no yes no yes     no yes      no yes
credit_rating                                      
excellent       3   3  3   3      3   3       3   3
fair            2   6  2   6      2   6       2   6

In [12]:
entropy_buy_computer_credit_rating = (8/14)*entropy([2/8, 6/8], base=2)+(6/14)*entropy([3/6, 3/6], base=2)
entropy_buy_computer_credit_rating

0.8921589282623617

In [13]:
gain_buy_computer_age = entropy_buy_computer - entropy_buy_computer_age
gain_buy_computer_age

0.246749819774439

In [14]:
gain_buy_computer_income = entropy_buy_computer - entropy_buy_computer_income
gain_buy_computer_income

0.029222565658954758

In [15]:
gain_buy_computer_student = entropy_buy_computer - entropy_buy_computer_student
gain_buy_computer_student

0.15183550136234159

In [16]:
gain_buy_computer_credit_rating = entropy_buy_computer - entropy_buy_computer_credit_rating
gain_buy_computer_credit_rating

0.04812703040826938

In [17]:
#Den höchsten Wert hat das Alter, deshalb sollte zuerst danach getrennt werden

In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
 
converted_data = train_data.apply(preprocessing.LabelEncoder().fit_transform)
converted_data

,id,age,income,student,credit_rating,Buy_Computer
0,0,2,0,0,1,0
1,1,2,0,0,0,0
2,2,0,0,0,1,1
3,3,1,2,0,1,1
4,4,1,1,1,1,1
5,5,1,1,1,0,0
6,6,0,1,1,0,1
7,7,2,2,0,1,0
8,8,2,1,1,1,1
9,9,1,2,1,1,1


In [19]:
target = converted_data["Buy_Computer"].to_numpy()
converted_data = converted_data[["age","income","student","credit_rating"]].to_numpy()

In [20]:
decision_tree = DecisionTreeClassifier(max_depth=4)
decision_tree.fit(converted_data,target)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=4, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [21]:
from sklearn.tree import export_graphviz
 
export_graphviz(
    decision_tree,
    out_file="C:\\Users\Susi\Desktop\Jupyter\Graphviz\computer.dot",
        feature_names=["Age","Income","Student","Credit Rating"],
        class_names=["Yes","No"],
        rounded=True,
        filled=True
)
#Der decision-tree zeigt, dass auch sklearn zuerst nach dem Alter trennt